# Notebook s1_keywords pour extraire uniquement les mots-clés relatifs à 1940 (Extraction des mots-clés)

In [2]:
########### importation des library #####################
import os
import yake

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50) #la langue c'est du francais
kw_extractor

In [4]:
# Lister les Fichiers
data_path = "../data/txt/"
files = os.listdir(data_path)

In [ ]:
# Imprimer le nombre de fichiers identifiés
len(files)

In [ ]:
# Les 2 premiers fichiers
files[:2]

In [ ]:
####### selection de tous les  fichiers de 1940 et affichage
this_files = [F for F in files if '1940' in F]
this_files

In [ ]:
# Récupération du  texte dans les fichiers (Cette boucle va lire chaque fichier dans this_files, ajouter son contenu à la variable all_texts, et enfin, le print à la fin affiche les 500 premiers caractères du contenu combiné.)

all_texts = ""

for file_name in this_files:
    file_path = os.path.join(data_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as F:
        all_texts += F.read()

print(all_texts[:500])


In [ ]:
# Extraction des mots clés de notre fichiers de fusion all_texts
keywords = kw_extractor.extract_keywords(all_texts)
keywords

In [ ]:
# Ne garder que les tigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 3:
        kept.append(kw)
kept

In [ ]:
# Liste des mots-clés composés de deux mots pour chaque fichier trié de this_files et affichage des sources dans lesquels ces mots-clés sont mentionnés.
for F in sorted(this_files):
    all_texts = open(os.path.join(data_path, F), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(all_texts)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{F} mentions these keywords: {', '.join(kept)}...")

# Notebook Question 2 wordcloud.ipynb

## Représentation Nuages de mots (question de recherche: Aperçu des thèmes les plus courants discutés dans les journaux Belge en 1940)

In [ ]:
#Importation des librairies nécessaires
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

In [13]:
# Stopwords 
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", "jeune", "nouveau","partie","heures","très",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous","faire", "être","voir","jours","rien","beau","pet","etc","minutes","devant","jard","suite","chez","divers", "avoir", "dire", "voit","vie","pierre","cas","alors","soir","tant","mme","pied","bas","près","vis","chaque", "dès",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non","jean", "paul", "dame", "monsieur",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres","bon", "grand", "petit", "bien", "gros",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem","des","bien","un","une","à","dans","vers", "point", "lieu","Rue","leurs","quelques","elles","toute","déjà","puis","celui","quand","avant","elles"]
sw = set(sw)

### enrichissement de la liste de nos stopword sw
journal_stopwords = ["titre", "paragraphe", "section", "source", "référence", "aujourd'hui", 
                         "hier", "demain", "actuellement", "récemment", "précédemment", "information", 
                         "rapport", "article", "annonce", "déclaration", "commentaire", "peut-être", 
                         "probablement", "possible", "éventuellement", "journaliste", "rédacteur", 
                         "correspondant", "spécialiste", "expert", "publié", "mis_à_jour", "édition", "nuit",
                         "version"]

### mot vide relatif a la négation
negation_words = ["ni", "aucun", "néanmoins", "quoique", "bien_que"]

#mot vide relatif aux temps 
temps_words = ["toujours", "jamais", "souvent", "rarement", "parfois"]

### mot vide relatif aux quantité 
quantity_words = ["beaucoup", "plusieurs", "première", "dernière", "autant"]

#### mot vide relatif au opignons et adverbe 
additional_words = ["vraiment", "plutôt", "assez", "trop", "peu","même", "notamment", "parce", "car", "puisque"]
sw.update(additional_words)


sw.update(quantity_words)
sw.update(temps_words)
sw.update(negation_words)
sw.update(journal_stopwords)

In [14]:
#######################
# selection d'une année
year = 1940

In [ ]:
# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]
len(txts)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r', encoding='utf-8') as f:
        content_list.append(f.read())
        
# Compter le nombre d'éléments (=fichiers) dans la liste
len(content_list)

In [ ]:
# Imprimer les 200 premiers caractères du contenu du premier fichier
content_list[0][0:200]

In [18]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'{year}.txt'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:500]

In [20]:
## Nettoyage du fichier à l'aide d'une fonction de nettoyage

def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
### Appliquer la fonction sur le fichier complet de l'année

clean_text(year, folder=temp_path)

In [ ]:
# Vérification du résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

In [ ]:
### Affichage des termes les plus fréquents
frequencies = Counter(after.split()) #counter donne un compteur par mots afin d'evaluer le nombre de fois kil apparait 
print(frequencies.most_common(10)) 

In [ ]:
### Créer, stocker et afficher le nuage de mots (pensé a ce que l'on a fait et ce quon a mis en oeuvre afin que le nuage de mot soi le plus precis et beau possible)

cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))

Image(filename=os.path.join(temp_path, f"{year}.png"))

# Question:3 Extraction d'entités nommées principales (personnes, organisations et lieux) pertinentes pour 1940

In [25]:
### importation des librairies
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

In [26]:
nlp = spacy.load('fr_core_news_lg')

In [ ]:
# Imprimer le corpus de Spacy
sentences

In [ ]:
# Isoler la première phrase
sent = sentences[0]
sent

In [29]:
# Traiter la phrase avec Spacy
doc = nlp(sent)

In [ ]:
type(doc) ###sur l'objet doc on imprime l'attribut text (doc document enrichi par spacy

In [ ]:
doc.text

In [ ]:
doc.to_json() ### il analyse le mot en commancan par l'indice 0 ( Appel quest que ces: tag nom propre , etc..

In [ ]:
# Appliquer le test sur toutes les phrases
for sent in sentences: # on itere atraver la liste des phrase 
    doc = nlp(sent) # on appilique sur chaquune de ces phrase les modele pour extraire les entité
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}") 
    else:
        print(f"'{doc.text}' ne contient aucune entité")

In [34]:
## Appliquer la reconnaissance d'entités nommées sur notre corpus de 1940 nomée 1940.txt ds le dossier tmp
# Charger le texte
n=100000
text = open("../data/tmp/1940.txt", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

In [ ]:
### comtapbilisation et affichage des entités
entities = defaultdict(int)

for ent in doc.ents:
    if ent.label_ in ["PER", "ORG", "LOC"] and len(ent.text.strip()) > 3:
        entities[(ent.text.strip(), ent.label_)] += 1  # Comptabiliser chaque entité avec son label

# Trier les entités par fréquence
sorted_entities = sorted(entities.items(), key=lambda kv: kv[1], reverse=True)

# Imprimer les 10 premières entités et leurs fréquences
for (entity, label), freq in sorted_entities[:10]:
    print(f"{entity} [{label}] apparait {freq} fois dans le corpus")


# Question 4 : extraction de 10 phrases aléatoires dans les articles de 1940 et analysez-les à l’aide du notebook s4_sentiment.ipynb

In [37]:
##import des lib
import sys
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [38]:
####### Création d'une fonction `get_sentiment` qui évaluera le sentiment et la subjectivité de nos phrases

tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

### Sélection et extraction aléatoire des phrases de notre corpus 1940.txt

In [ ]:
####################### Sélection et extraction aléatoire des phrases de notre corpus ####################
import os
import random


def select_random_sentences(file_path, num_sentences=10):
    with open(file_path, 'r', encoding='latin-1') as file:
        text = file.read()

    # Diviser le texte en phrases
    sentences = text.split('.')
    
    # verifie/ s'assurer qu'il y a assez de phrases
    if len(sentences) < num_sentences:
        return "Il n'y a pas assez de phrases dans le fichier."
    
    # Sélectionner aléatoirement des phrases (10)
    selected_sentences = random.sample(sentences, num_sentences)
    
    return selected_sentences

# chemin d'accès au fichier
random_sentences = select_random_sentences('../data/tmp/1940.txt')

for i, sentence in enumerate(random_sentences):
    print(f"Sentence {i+1}: {sentence.strip()}")



#### Résultat sélectionné après avoir exécuté plusieurs fois la cellule ci-haut pour choisir des phrase avec sens:
#### Phrases sélectionner pour analyse:  

##### 1. Le docteur Laurent de Dampremy nous prie de rectifier : il donne des renseignements sur le 7e chasseurs à pied, 1er bataillon et non sur les 2e et 3e bataillons.
##### 2. Un blondinet de cinq ans avait rendu visite au Royaume des Jouets, en compagnie de sa tante.
##### 3. Au Palais des Sports d'Anvers, Schulte a battu Clautier en match poursuite.
##### 4. Neckerman était aussi un pauvre homme à la recherche de l'oubli des tracas d'une journée de laborieux travail.
##### 5. En Italie, le problème juif est resté latent pendant des années, car il n'y avait pas atteint cette importance et cette urgence qui le caractérisaient auprès d'autres nations européennes
##### 6. La neutralité ainsi conçue devient un élément d'équilibre et de barrage à un endroit crucial de l'Europe.
##### 7. L'ennemi a continué ses attaques aériennes sur les villes de la Ligurie et de la Sardaigne.
##### 8.Des cannibales armés de lances et de boucliers accouraient d'un village proche dont on apercevait les paillotes.
##### 9. Tous les soirs, la place de la station est noire de monde ; femmes, enfants et parents attendent le retour des absents
##### 10. Du point de vue sportif, le résultat de la rencontre Liersche-Beerschot est, sans conteste, le plus intéressant.

#### Analyser le sentiment d'une phrase

In [ ]:
# 1
get_sentiment("Le docteur Laurent de Dampremy nous prie de rectifier : il donne des renseignements sur le 7e chasseurs à pied, 1er bataillon et non sur les 2e et 3e bataillons.")

In [ ]:
# 2
get_sentiment("Un blondinet de cinq ans avait rendu visite au Royaume des Jouets, en compagnie de sa tnte.")

In [ ]:
# 3
get_sentiment("Au Palais des Sports d'Anvers, Schulte a battu Clautier en match poursuite.")

In [ ]:
# 4
get_sentiment("Neckerman était aussi un pauvre homme à la recherche de l'oubli des tracas d'une journée de laborieux travail.")

In [ ]:
#5
get_sentiment("En Italie, le problème juif est resté latent pendant des années, car il n'y avait pas atteint cette importance et cette urgence qui le caractérisaient auprès d'autres nations européennes")

In [ ]:
# 6
get_sentiment("La neutralité ainsi conçue devient un élément d'équilibre et de barrage à un endroit crucial de l'Europe.")

In [ ]:
# 7
get_sentiment("L'ennemi a continué ses attaques aériennes sur les villes de la Ligurie et de la Sardaigne.")

In [ ]:
# 8
get_sentiment("Des cannibales armés de lances et de boucliers accouraient d'un village proche dont on apercevait les paillotes.")

In [ ]:
# 9
get_sentiment("Tous les soirs, la place de la station est noire de monde ; femmes, enfants et parents attendent le retour des absents")

In [ ]:
# 10
get_sentiment("Du point de vue sportif, le résultat de la rencontre Liersche-Beerschot est, sans conteste, le plus intéressant")